In [6]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2024-03-09 18:29:56--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt’

dbpedia_sample_abst 100%[===================>]   5.92M  17.1MB/s    in 0.3s    

2024-03-09 18:29:57 (17.1 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt’ saved [6208417/6208417]



In [7]:
!head -n 2 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and


In [9]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt"

In [11]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')

In [13]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[:2000]]
stopwords = list(stop_words.words('english'))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [18]:
tp = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 75.9kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 9.59MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 154kB/s]
config.json: 100%|██████████| 571/571 [00:00<00:00, 1.76MB/s]
/Users/borosabel/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
pytorch_model.bin: 100%|██████████| 438M/438M [00:23<00:00, 18.3MB/s] 
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 252kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.62MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 5.29MB/s]
Batches: 100%|██████████| 10/10 [00:34<00:00,  3.41s/it]


In [20]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset)

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac